In [13]:
def evaluate_accuracy(data_iter, net1, net2, device):
#     if device is None and isinstance(net1, torch.nn.Module):
#         # 如果没指定device就使用net的device
#         device = list(net1.parameters())[0].device 
    acc_sum, n = 0.0, 0
    net1.eval()
    net2.eval()
    with torch.no_grad():
        for X, y in data_iter:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds1 = net1(X.to(device))
            preds2 = net2(X.to(device))
            preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            
            acc_sum += (torch.from_numpy(preds).to(device).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            
#             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
#             net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n

In [6]:
def evaluate_accuracy2(data_iter, net, device):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
           
            net.eval() # 评估模式, 这会关闭dropout
            acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
            net.train() # 改回训练模式
            n += y.shape[0]
    return acc_sum / n

In [7]:
evaluate_accuracy2(test_iter, resnet50, device)

0.9180327868852459

In [1]:
%matplotlib inline
import torch
import time
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os
from scipy.special import softmax

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [11]:
def load_data_face_50(batch_size):
    transform1 = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(0),
        torchvision.transforms.Resize([330,330]),
        torchvision.transforms.CenterCrop([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])
    transform2 = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(1),
        torchvision.transforms.Resize([330,330]),
        torchvision.transforms.CenterCrop([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])

    test_imgs1 = torchvision.datasets.ImageFolder('dataset/test', transform=transform1)
    test_imgs2 = torchvision.datasets.ImageFolder('dataset/test', transform=transform2)
#     train_iter = torch.utils.data.DataLoader(train_imgs, batch_size=batch_size, shuffle=True, num_workers=4)
    test_iter1 = torch.utils.data.DataLoader(test_imgs1, batch_size=batch_size, shuffle=False, num_workers=4)
    test_iter2 = torch.utils.data.DataLoader(test_imgs2, batch_size=batch_size, shuffle=False, num_workers=4)
    return test_iter1,test_iter2

In [2]:
def load_data_face_18(batch_size):
    transform1 = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(0),
#         torchvision.transforms.Resize([330,330]),
        torchvision.transforms.Resize([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])
    
    transform2 = torchvision.transforms.Compose([
       # torchvision.transforms.Grayscale(num_output_channels=1), # 彩色图像转灰度图像num_output_channels默认1
        torchvision.transforms.RandomHorizontalFlip(1),
#         torchvision.transforms.Resize([330,330]),
        torchvision.transforms.Resize([224, 224]),
        
        torchvision.transforms.ToTensor()
    ])

    test_imgs1 = torchvision.datasets.ImageFolder('dataset/test', transform=transform1)
    test_imgs2 = torchvision.datasets.ImageFolder('dataset/test', transform=transform2)

    test_iter1 = torch.utils.data.DataLoader(test_imgs1, batch_size=batch_size, shuffle=False, num_workers=4)
    test_iter2 = torch.utils.data.DataLoader(test_imgs2, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return test_iter1, test_iter2

In [3]:
resnet18 = torch.load('./resnet18.pkl').to(device)

In [13]:
resnet50 = torch.load('./resnet50918.pkl').to(device)

In [15]:
batch_size = 32
test_iter_501, test_iter_502 = load_data_face_50(batch_size)
# test_iter_181, test_iter_182 = load_data_face_18(batch_size) 

In [28]:
# def evaluate_accuracy(data_iter_18, data_iter_50, net1, net2, device):
# #     if device is None and isinstance(net1, torch.nn.Module):
# #         # 如果没指定device就使用net的device
# #         device = list(net1.parameters())[0].device 
#     acc_sum, n = 0.0, 0
#     net1.eval()
#     net2.eval()
# #     preds_18 = []
# #     preds_50 = []
#     with torch.no_grad():
#         for X1,  X2 in enumerate(data_iter_18, data_iter_50):
#             for x, y in X1:
#                 print(x.shape)
# #             net1.eval() # 评估模式, 这会关闭dropout
#             preds18 = net1(X1.to(device))
#             preds50 = net2(X2.to(device))
# #             preds18 = softmax(preds_18.cpu().numpy())
#             preds = 0.5 * softmax(preds18.cpu().numpy()) + 0.5 * softmax(preds50.cpu().numpy()) 
# #             preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
#             acc_sum += (torch.from_numpy(preds).to(device).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
# #             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
# #             net.train() # 改回训练模式
#             n += y1.shape[0]
#     return acc_sum / n

In [6]:
def evaluate_accuracy(data_iter_181, data_iter_182, net1, net2, device):
#     if device is None and isinstance(net1, torch.nn.Module):
#         # 如果没指定device就使用net的device
#         device = list(net1.parameters())[0].device 
    acc_sum, n = 0.0, 0
    net1.eval()
    net2.eval()
    preds_18 = []
#     preds_50 = []
    with torch.no_grad():
        for X1, y1 in data_iter_181:
#             net1.eval() # 评估模式, 这会关闭dropout
            preds18 = net1(X1.to(device))
#             preds2 = net2(X.to(device))
            preds18 = softmax(preds18.cpu().numpy())
#             preds = 0 * softmax(preds1.cpu().numpy()) + 1 * softmax(preds2.cpu().numpy())
            preds_18.append(preds18)
        m = 0
        for X2, y2 in data_iter_182:
            preds50 = net1(X2.to(device))
#             preds2 = net2(X.to(device))
            preds50 = softmax(preds50.cpu().numpy())
            preds = 0.5 * preds50 + 0.5 * preds_18[m]
            acc_sum += (torch.from_numpy(preds).to(device).argmax(dim=1) == y2.to(device)).float().sum().cpu().item()
            m += 1
#             acc_sum += (net1(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
#             net.train() # 改回训练模式
            n += y2.shape[0]
    return acc_sum / n

In [7]:
from scipy.special import softmax

In [ ]:
test_acc = evaluate_accuracy(test_iter_501, test_iter_502, resnet50, resnet50, device)
print(test_acc)

In [1]:
from scipy.special import softmax